In [1]:
import pandas as pd
import gc
sunat_train=pd.read_csv('../data/intermediate/sunat_train.csv', index_col = 'key_value')
sunat_test=pd.read_csv('../data/intermediate/sunat_test.csv', index_col = 'key_value')
censo_train=pd.read_csv('../data/intermediate/censo_train.csv', index_col = 'key_value')
censo_test=pd.read_csv('../data/intermediate/censo_test.csv', index_col = 'key_value')
se_train=pd.read_csv('../data/intermediate/se_train.csv', index_col = 'key_value')
se_test=pd.read_csv('../data/intermediate/se_test.csv', index_col = 'key_value')
rcc_train=pd.read_csv('../data/intermediate/rcc_train.csv', index_col = 'key_value')
rcc_test=pd.read_csv('../data/intermediate/rcc_test.csv', index_col = 'key_value')
y_train = pd.read_csv('../data/y_train.csv', index_col = 'key_value')

In [2]:
train = rcc_train.join(sunat_train).join(censo_train).join(se_train)
test = rcc_test.join(sunat_test).join(censo_test).join(se_test)
del rcc_train, sunat_train, censo_train, se_train, rcc_test, sunat_test, censo_test, se_test

In [4]:
# !pip install catboost
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = CatBoostClassifier(n_estimators=1000,  eval_metric = 'AUC', max_depth = 6)
    learner.fit(Xt, yt,  early_stopping_rounds=10, 
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) # cat_features = var_categ_index,
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
    gc.collect()
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))

********** 0 **********
Learning rate set to 0.128212
0:	test: 0.7801722	test1: 0.7869643	best: 0.7869643 (0)	total: 714ms	remaining: 11m 52s
50:	test: 0.8471719	test1: 0.8468172	best: 0.8468172 (50)	total: 33.3s	remaining: 10m 20s
100:	test: 0.8569771	test1: 0.8549128	best: 0.8549128 (100)	total: 1m 6s	remaining: 9m 48s
150:	test: 0.8639222	test1: 0.8601417	best: 0.8601417 (150)	total: 1m 36s	remaining: 8m 59s
200:	test: 0.8685928	test1: 0.8628305	best: 0.8628305 (200)	total: 2m 5s	remaining: 8m 20s
250:	test: 0.8722389	test1: 0.8645643	best: 0.8645643 (250)	total: 2m 35s	remaining: 7m 43s
300:	test: 0.8749598	test1: 0.8657718	best: 0.8657724 (299)	total: 3m 3s	remaining: 7m 6s
350:	test: 0.8772226	test1: 0.8664093	best: 0.8664093 (350)	total: 3m 31s	remaining: 6m 30s
400:	test: 0.8793934	test1: 0.8670013	best: 0.8670013 (400)	total: 3m 58s	remaining: 5m 55s
450:	test: 0.8814293	test1: 0.8673978	best: 0.8673978 (450)	total: 4m 25s	remaining: 5m 22s
500:	test: 0.8833965	test1: 0.867771

In [5]:
fi.sort_values(ascending=False).head(50)#.plot.bar(figsize=(15,10))

COD_CLASIFICACION_DEUDOR_nunique_ult1mes    0.128444
PRODUCTO_1_ult1meses                        0.051593
COD_CLASIFICACION_DEUDOR_nunique_ult2mes    0.032101
COD_CLASIFICACION_DEUDOR_5_ult3meses        0.023801
PRODUCTO_3_ult3meses                        0.022492
RIESGO_DIRECTO_1_ult1meses                  0.021603
cod_instit_financiera_32_ult1meses          0.019059
tipo_credito_11_ult1meses                   0.018736
tipo_credito_12_ult1meses                   0.018087
PRODUCTO_0_ult6meses                        0.017963
PRODUCTO_1_ult3meses                        0.017600
PRODUCTO_1_ult12meses                       0.015989
acteco18                                    0.014268
tipo_credito_11_ult12meses                  0.012350
edad                                        0.012240
COD_CLASIFICACION_DEUDOR_max_ult6mes        0.011717
tip_lvledu                                  0.011404
cod_instit_financiera_33_ult1meses          0.010875
acteco15                                    0.

In [6]:
test_probs.name = "target"
test_probs.to_csv("../results/test8.csv")

In [3]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np

In [4]:
learner_ = CatBoostClassifier(n_estimators=1000, eval_metric = 'AUC', max_depth = 6)
learner_.fit(train, y_train,  early_stopping_rounds=10, verbose = 50)

Learning rate set to 0.126968
0:	total: 457ms	remaining: 7m 36s
50:	total: 18.6s	remaining: 5m 45s
100:	total: 36s	remaining: 5m 20s
150:	total: 52.6s	remaining: 4m 55s
200:	total: 1m 8s	remaining: 4m 33s
250:	total: 1m 24s	remaining: 4m 12s
300:	total: 1m 40s	remaining: 3m 52s
350:	total: 1m 54s	remaining: 3m 31s
400:	total: 2m 9s	remaining: 3m 12s
450:	total: 2m 24s	remaining: 2m 55s
500:	total: 2m 38s	remaining: 2m 38s
550:	total: 2m 53s	remaining: 2m 21s
600:	total: 3m 8s	remaining: 2m 5s
650:	total: 3m 22s	remaining: 1m 48s
700:	total: 3m 37s	remaining: 1m 32s
750:	total: 3m 51s	remaining: 1m 16s
800:	total: 4m 5s	remaining: 1m 1s
850:	total: 4m 20s	remaining: 45.5s
900:	total: 4m 34s	remaining: 30.2s
950:	total: 4m 49s	remaining: 14.9s
999:	total: 5m 3s	remaining: 0us


In [5]:
fi_ = pd.Series(learner_.feature_importances_ / learner_.feature_importances_.sum(), index=train.columns)
fi_.sort_values(ascending=False).head(50)

COD_CLASIFICACION_DEUDOR_nunique_ult1mes    0.076290
PRODUCTO_1_ult1meses                        0.051526
COD_CLASIFICACION_DEUDOR_nunique_ult2mes    0.046709
acteco4                                     0.030643
PRODUCTO_3_ult3meses                        0.023702
dehogar8                                    0.022259
mattec6                                     0.021782
tipo_credito_12_ult1meses                   0.020019
RIESGO_DIRECTO_1_ult1meses                  0.019946
COD_CLASIFICACION_DEUDOR_5_ult3meses        0.019235
COD_CLASIFICACION_DEUDOR_max_ult6mes        0.019191
PRODUCTO_0_ult6meses                        0.018268
PRODUCTO_1_ult12meses                       0.018167
dehogar2                                    0.016858
cod_instit_financiera_32_ult1meses          0.014608
PRODUCTO_1_ult3meses                        0.014428
tipo_credito_11_ult1meses                   0.014054
tipo_credito_11_ult12meses                  0.013649
hig4                                        0.

In [6]:
test_probs_ = pd.Series(learner_.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_.shape

(396666,)

In [8]:
test_probs_.name = "target"
test_probs_.to_csv("../results/test8_v2.csv")